In [8]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [9]:
from django.db.models import Q
from copy import deepcopy
from openpyxl import load_workbook
from openpyxl import Workbook
from datetime import datetime
from tqdm.notebook import tqdm
from django.contrib.gis.geos import Point
import sys
from django.contrib.gis.geos import GeometryCollection
import json
from django.contrib.admin.utils import flatten

In [10]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history

In [11]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [12]:
def transform_coordinates(x, y, srid=31982):

    pnt = Point(x, y, srid=srid)
    pnt.transform(4326)
    
    lng = pnt.coords[1]
    lat = pnt.coords[0]
    
    return Point(lat, lng)

In [1]:
company=Company.objects.get(name__icontains='GES - UHE-ESTREITO')
user=User.objects.get(username='engie')
occurrence_type=OccurrenceType.objects.get(uuid='f200613b-8862-4a24-8c16-d6babd215fd7')
species_type=OccurrenceType.objects.get(uuid='c16b1d6b-856c-4ff7-ae2f-f68d5223a077')
operational_control=OperationalControl.objects.get(uuid='b4295792-fd8c-4ff8-b603-a427eb689d82')
filename='Matrizes-FAPEAD (1) (1)'
occurrence_type.name,species_type.name,operational_control

('Cadastro de Matriz Vegetal',
 'Cadastro',
 <OperationalControl: [GES - UHE-ESTREITO]: 701 - [Serviços de desenvolvimento de plataforma para gestão de equipes e serviços ambientais]>)

In [ ]:
# ocr=OccurrenceRecord.objects.filter(operational_control=operational_control,
#                 company=company,
#                 occurrence_type=occurrence_type)
# ocr.delete()
# ocr.count()

In [18]:
wb = load_workbook(filename + ".xlsx")
sheetname = wb.sheetnames[0]
ws = wb[sheetname]
ws

<Worksheet "matrizes atualizadas">

In [19]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [20]:
for a in values:
    
    if isinstance(a['CAP (m)'],str):
        valor = a['CAP (m)'].replace(',','.')
        a['CAP (m)'] =sum([float(a) for a in valor.split('+')])
    if a['Nome Popular'].strip() == 'Olho-de-Boi':
        a['Nome Popular']='Olho de Boi'
    if a['Nome Popular'].strip() == 'Embauba':
        a['Nome Popular']='Embaúba'

In [53]:
transform_coordinates(values[3]['Coord. X'],values[3]['Coord. Y'])

In [14]:
values[3]

{'Nº': 4,
 'Nome Popular': 'Tarumã',
 'Nome científico': 'Vitex polygama Cham.',
 'Coord. X': 229587,
 'Coord. Y': 9271695,
 'CAP (m)': 1.91,
 'Altura total (m)': 16,
 'Município ': 'Estreito',
 'Estado': 'MA',
 'Fase fenológica': 'Os frutos já haviam caído a um mês',
 'Data da demarcação': datetime.datetime(2013, 5, 13, 0, 0),
 'Condições Fitossanitárias  (Ótimo/Regular/Ruim)': None,
 'Sementes coletadas (kg)': None}

In [75]:
# st = 'Caesalpinia pyramidalis Tul'
# ' '.join(st.split(' ')[0:2])
v=values[808]
OccurrenceRecord.objects.filter(Q(form_data__popular_name__icontains=v['Nome Popular'].strip()) | 
                                Q(form_data__species__icontains=' '.join(v['Nome científico'].split(' ')[0:2])),
                                company=company,occurrence_type=species_type).first().uuid

AttributeError: 'NoneType' object has no attribute 'uuid'

In [88]:
v['Nome Popular'],' '.join(v['Nome científico'].split(' ')[0:2])
datetime.today()

datetime.datetime(2023, 9, 4, 11, 44, 36, 380334)

In [21]:
objects=[]

for index,a in enumerate(tqdm(values)):
    cientifico = ' '.join(a['Nome científico'].split(' ')[0:2])
    try:
                
        properties=[]
        point=transform_coordinates(float(a['Coord. X']), float(a['Coord. Y']),srid=31983)
        geometry=GeometryCollection()
        geometry.append(point)
        properties.append({})


        
        objects.append(OccurrenceRecord(
                    operational_control=operational_control,
                    company=company,
                    occurrence_type=occurrence_type,
                    geometry=geometry,
                    created_by=user,
                    properties=properties,
                    datetime=datetime.today(),
#                     other_reference=a['Outra referência'] if a['Outra referência'] is not None else "",
                    
                    form_data={'records': str(
                        OccurrenceRecord.objects.filter(Q(form_data__popular_name__icontains=a['Nome Popular'].strip()) | 
                                                                              Q(form_data__species__icontains=cientifico),
                                                                              company=company,occurrence_type=species_type).first().uuid),
                               'specie_type': '1',
                               'matrix_serial': a['Nº'],
                               'matrix_size': round(float(a['Altura total (m)']), 2),
                               'demarcation_time': a['Data da demarcação'].strftime('%Y-%m-%dT%H:%M:00.000000-03:00') if a['Data da demarcação'] else None,
                               'phytosanitary_condition': get_value('Condições fitossanitárias', occurrence_type, a['Condições Fitossanitárias']) if a['Condições Fitossanitárias'] != None else None,
                              'phenological_stage': a['Fase fenológica'].strip() if a['Fase fenológica'] else "",
                              'collected_seeds': a['Sementes coletadas (kg)'] if a['Sementes coletadas (kg)'] else None,
                              'matrix_cap': round(float(a['CAP (m)']), 2)}))
    
    
    except Exception as e:
        print(f"{index+2} {a['Nome Popular']}, {cientifico}{e}")

  0%|          | 0/840 [00:00<?, ?it/s]

221 Tingui, Magonia pubescens'ò'


In [90]:
objects[839].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f25e1d80810>,
 'uuid': UUID('03cd3050-155d-44cf-94ec-a522b446fe08'),
 'datetime': datetime.datetime(2023, 9, 4, 11, 45, 0, 772532),
 'number': '',
 'created_at': None,
 'updated_at': None,
 'reviews': 0,
 'company_id': UUID('42ddc389-3258-4daf-868a-097235427e0f'),
 'editable': True,
 'is_approved': False,
 'uf_code': None,
 'city_id': None,
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': <GeometryCollection object at 0x7f25e015f130>,
 'properties': [{}],
 'distance_from_dam': 0,
 'other_reference': '',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'search_tag_description': '',
 'occurrence_type_id': UUID('f200613b-8862-4a24-8c16-d6babd215fd7'),
 'form_data': {'records': '67760d49-f3c3-4b75-aad2-45e46737a86d',
  'specie_type': '1',
  'matrix_serial': 851,
  'matrix_size': 10.0,
  'dema

In [22]:
for a in tqdm(objects):
    a.save()

  0%|          | 0/839 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.235497) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.235497) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.260837) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.505704) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.505704) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.527768) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.756416) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.756416) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.772331) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.998703) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:49.998703) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:20:50.022126) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

...

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:02.545250) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:02.545250) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:02.578521) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:02.863418) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:02.863418) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:02.884386) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.178158) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.178158) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.199855) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.499327) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.499327) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.521202) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.829980) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.829980) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-15 10:21:03.853331) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

In [23]:
ocr=OccurrenceRecord.objects.get(uuid='d84d7341-f458-415b-943a-4226ef2ba19f')
ocr

<OccurrenceRecord: [GES - UHE-ESTREITO] - UHET-RG-2023.2351 - 2023-09-15 13:20:56.428704+00:00>

In [30]:
point=transform_coordinates(229665, 9271697,srid=31983)
geometry=GeometryCollection()
geometry.append(point)
ocr.geometry = geometry
ocr.save()

In [ ]:
ocr.geometry

In [2]:
wb = load_workbook('Matrizes Georreferrenciadas do Horto UHE Ita (completa)' + ".xlsx")
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

In [3]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [4]:
names=[]
jason={}
# counter=1

for a in values:

    names.append([{"name": a['Nome Comum:'], "value": a['Nome Cientifíco']}])

In [5]:
names=flatten(names)

In [6]:
with open('names'+'_if'+'.txt', 'w') as outfile:  
    json.dump(names, outfile, indent = 4)

# Cadastro de Espécies

In [14]:
import re

In [70]:
# Ita, Cana Brava, Estreito,Jaguara, machadinho, passo fundo,ponte de pedra,salto osorio, salto santiago, são salvador
company=Company.objects.get(name='GES - UHE-SÃO SALVADOR')
occurrence=OccurrenceType.objects.get(company=company, occurrence_kind=next(a['value'] for a in company.custom_options['operationalControl']['fields']['kind']['selectOptions']['options'] if a['name'] == 'Cadastro de espécies'))
user=User.objects.get(username='engie')
operational=OperationalControl.objects.get(uuid='9948c88b-b4bc-4c14-80d4-239f6a25d452')
occurrence,operational

(<OccurrenceType: Cadastro - ['GES - UHE-MIRANDA', 'GES - UHE-PASSO FUNDO', 'GES - UHE-SALTO OSORIO', 'GES - UHE-PONTE DE PEDRA', 'GES - UHE-SALTO SANTIAGO', 'GES - UHE-JAGUARA', 'GES - UHE-ITA', 'GES - UHE-ESTREITO', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-MACHADINHO', 'GES - UHE-CANA BRAVA']>,
 <OperationalControl: [GES - UHE-ITA]: 709 - [Serviços de desenvolvimento de plataforma para gestão de equipes e serviços ambientais]>)

In [71]:
filename='Lista de Todas as plantas dos Horto'

In [72]:
wb = load_workbook(filename+'.xlsx')
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

In [73]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [74]:
len(values),re.split(r',|-|/| ', values[3]['Nome Popular'].strip())

(753, ['ABRICÓ', 'DE', 'MACACO'])

In [75]:
v=values[244]
popular_splited = re.split(r',|-|/| ',v['Nome Popular'].strip())
species_splited = re.split(r',|-|/',v['Nome Científico'].strip())
OccurrenceRecord.objects.filter(
                                        Q(form_data__popular_name=v['Nome Popular'].strip()) | 
                                        Q(form_data__species=v['Nome Científico'].strip()) |
                                           (Q(form_data__popular_name__icontains=popular_splited[0]) &
                                           Q(form_data__species__icontains=species_splited[0])),company=company),v

(<QuerySet [<OccurrenceRecord: [GES - UHE-SÃO SALVADOR] - UHSA-RG-2023.0077 - 2023-08-07 03:00:00+00:00>]>,
 {'Nome Popular': 'Cincho \xa0', 'Nome Científico': 'Sorocea bonplandii'})

In [76]:
reps=[]
existe,nexiste=0,0
for i,a in enumerate(tqdm(values)):
    try:
        popular_splited = re.split(r',|-|/| ',a['Nome Popular'].strip())
        species_splited = re.split(r',|-|/| ',a['Nome Científico'].strip())
        ocr=OccurrenceRecord.objects.filter(
                                        Q(form_data__species=a['Nome Científico'].strip()) &
                                        Q(form_data__popular_name__icontains=popular_splited[0]),company=company)
        if not ocr.exists():
            reps.append(OccurrenceRecord(occurrence_type=occurrence,
                                company=company,
                                operational_control=operational,
                                datetime=datetime.today(),
                                created_by=user,
                                firm=Firm.objects.filter(company=company,name='MEIO AMBIENTE').first(),
                                responsible=user,
                                form_data={
                                    'kingdom': '2',                                    
                                    'popular_name': a['Nome Popular'].strip(),
                                    'species': a['Nome Científico'].strip() if a['Nome Científico'] is not None else None
                                }))
            nexiste+=1
        else:
            existe+=1
#             print(popular_splited)
    except Exception as e:
        print(i+2,a['Nome Popular'],e,frf.of)

  0%|          | 0/753 [00:00<?, ?it/s]

In [79]:
existe,nexiste,reps[0].__dict__

(38,
 715,
 {'_state': <django.db.models.base.ModelState at 0x7f5f2214c390>,
  'uuid': UUID('215df39b-9570-4463-9a79-81df3c12ab73'),
  'datetime': datetime.datetime(2023, 10, 9, 21, 54, 40, 689754),
  'number': 'UHSA-RG-2023.0171',
  'created_at': datetime.datetime(2023, 10, 10, 0, 54, 56, 631370, tzinfo=<UTC>),
  'updated_at': datetime.datetime(2023, 10, 10, 0, 54, 56, 631390, tzinfo=<UTC>),
  'reviews': 0,
  'company_id': UUID('96bcaf1a-593b-4c6e-bdca-9b0dad0db0f1'),
  'editable': True,
  'is_approved': False,
  'uf_code': None,
  'city_id': None,
  'location_id': None,
  'place_on_dam': '',
  'river_id': None,
  'point': None,
  'geometry': None,
  'properties': [],
  'distance_from_dam': 0,
  'other_reference': '',
  'origin': None,
  'origin_media': None,
  'informer': {},
  'territorial_administration': '',
  'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
  'search_tag_description': '',
  'occurrence_type_id': UUID('c16b1d6b-856c-4ff7-ae2f-f68d5223a077'),
  'form_

In [78]:
for a in tqdm(reps):
    a.save()

  0%|          | 0/715 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:40.689754) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:40.689754) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:40.703982) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:40.893579) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:40.893579) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:40.906874) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:41.103334) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:41.103334) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:41.116933) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:41.329324) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:41.329324) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:41.343892) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

...

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:51.618853) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:51.618853) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:51.638637) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:51.830579) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:51.830579) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:51.844371) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.045048) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.045048) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.059484) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.258032) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.258032) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.272344) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.468390) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.datetime received a naive datetime (2023-10-09 21:54:52.468390) while time zone support is active.
  RuntimeWarning)


In [1]:
records=OccurrenceRecord.objects.filter(company=company, occurrence_type__uuid='f200613b-8862-4a24-8c16-d6babd215fd7')
records.count()

NameError: name 'company' is not defined

In [33]:
import uuid

def is_valid_uuid(val):
    try:
        uuid.UUID(str(val))
        return True
    except ValueError:
        return False

In [34]:
records_list=[]
counter=0

for a in tqdm(records):
    if not is_valid_uuid(a.form_data['popular_name']):
        try:
            species=OccurrenceRecord.objects.get(company=company, occurrence_type=occurrence,
                                            operational_control=operational, form_data__species=a.form_data['popular_name'])
            a.form_data['records']=str(species.uuid)
            records_list.append(a)
            counter+=1
        except Exception as e:
    #         print(str(species.uuid))
            print(a.form_data['popular_name'])

  0%|          | 0/1007 [00:00<?, ?it/s]

Sorocea bonplandii
Sorocea bonplandii
Diospyros coccolobifolia
Sorocea bonplandii
Plinia cauliflora
Sorocea bonplandii
Sorocea bonplandii
Plinia cauliflora
Sorocea bonplandii
Plinia cauliflora
Sorocea bonplandii
Sorocea bonplandii
Sorocea bonplandii
Sorocea bonplandii
Sorocea bonplandii


In [36]:
records_edit=bulk_update_with_history(records_list, OccurrenceRecord, user=user, use_django_bulk=True, batch_size=100)

In [82]:
occurrence=OccurrenceType.objects.get(company=company, occurrence_kind=next(a['value'] for a in company.custom_options['operationalControl']['fields']['kind']['selectOptions']['options'] if a['name'] == 'Cadastro de espécies'))
operational=OperationalControl.objects.get(firm__company=company,kind=next(a['value'] for a in company.custom_options['operationalControl']['fields']['kind']['selectOptions']['options'] if a['name'] == 'Cadastro de espécies'))
operational

<OperationalControl: [GES - UHE-SÃO SALVADOR]: 709 - []>

In [87]:
for b in tqdm(Company.objects.filter(name__startswith="GES")):
    operational=OperationalControl.objects.filter(firm__company=b,kind=next(a['value'] for a in company.custom_options['operationalControl']['fields']['kind']['selectOptions']['options'] if a['name'] == 'Cadastro de espécies'))
    if not operational.exists():
        print('skipping', b.name)
        continue
    oper = operational.first()
    OccurrenceRecord.objects.filter(occurrence_type=occurrence,
                                company=b,
                                operational_control=OperationalControl.objects.get(uuid='9948c88b-b4bc-4c14-80d4-239f6a25d452'),
                                created_at__gte='2023-10-9').update(
                                                                    operational_control=oper
                                                                    )

  0%|          | 0/14 [00:00<?, ?it/s]

skipping GES - PCH-RONDONÓPOLIS
skipping GES - PCH-JOSÉ GELÁZIO DA ROCHA
skipping GES - GEH


In [80]:
OccurrenceRecord.objects.filter(occurrence_type=occurrence,
                                company=company,
                                operational_control=operational,
                                created_at__gte='2023-10-9').count()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.created_at received a naive datetime (2023-10-09 00:00:00) while time zone support is active.
  RuntimeWarning)


715